In [1]:
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-8wozoiu_
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-8wozoiu_
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 121 kB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.7 MB 44.0 MB/s 
     |████████████████████████████████| 532 kB 47.9 MB/s 
     |████████████████████████████████| 139 kB 44.9 MB/s 
     |████████████████████████████████| 180 kB 47.9 MB/s 
     |████████████████████████████████| 97 kB 5.8 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for avalanche-lib: filename=avalanche_lib-0.0.1-py3-none-any.whl size=426447 sha256=a557d723d41872b93c14a929cb1ed5e1062a1556e1f7dce186ca7adf5

In [3]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.strategies import Naive

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [4]:
model = SimpleMLP(num_classes=10)


In [5]:
perm_mnist = PermutedMNIST(n_experiences=3)
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /root/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.avalanche/data/mnist/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()


In [8]:
cl_strategy = Naive(model, optimizer, criterion, train_mb_size=32, train_epochs=2,eval_mb_size=32, device=device)

In [10]:
results = []

In [11]:

for train_task in train_stream:
  cl_strategy.train(train_task, num_workers=4)
  results.append(cl_strategy.eval(test_stream))

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


100%|██████████| 1875/1875 [00:45<00:00, 41.27it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2054
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9413
100%|██████████| 1875/1875 [00:49<00:00, 38.23it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1718
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9503
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:06<00:00, 45.45it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1306
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9616
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 313/313 [00:06<00:00, 45.64it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task001/Exp001 = 2.6670
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.1547
-- S